We want to compare in computational terms Tridiagonal matrixes and Sparse matrixes with the "\" solver and other solving algorithms

In this first snippet of code we compared the "\" solver with a randomized 10000x10000 matrix with the 90% of zero-elements stored as a sparse one "A" and as a full one "B". We can see that in the first case it takes 5 times more time to find the u vector, and several more bytes of memory used. 

In [3]:
using SparseArrays
using Plots
using LinearSolve
using BenchmarkTools
using LinearAlgebra

function solver(Mat,b)
    if(size(Mat)[1]==size(Mat)[2])
        if(size(Mat)[1]!=size(b)[1])
            error("Wrong dimensions")
        end
        u = Mat \ b
        return u::Vector{Float64}
    else
        error("It's not a square matrix")
    end
end


n = 10000;
density = 0.1;
f = rand(n);
A = sprand(n, n, density);
display(@btime solver(A,f))
B = Matrix(A);
display(@btime solver(B,f));

10000-element Vector{Float64}:
 -0.021687341408287406
  0.46944385328899235
 -0.12072195669867251
 -2.444815301140367
 -0.04208179614342578
 -0.09024465672533925
 -0.43883308225588
  1.6177562586157486
 -0.5778069923092731
 -2.0666797895714746
  ⋮
 -0.6187286462893821
  1.3574887985304394
 -0.417606750025359
 -0.5944257960707379
  0.0013504531843248602
  0.07531418376453691
 -0.9019647948666345
 -1.1574250269410453
  1.025079034760371

  12.163 s (85 allocations: 3.81 GiB)

10000-element Vector{Float64}:
 -0.02168734135856396
  0.4694438533039351
 -0.12072195670892877
 -2.4448153011645894
 -0.04208179614074015
 -0.09024465672983147
 -0.43883308226916207
  1.6177562585984768
 -0.5778069923329632
 -2.0666797895278988
  ⋮
 -0.6187286462927292
  1.357488798538937
 -0.417606750024964
 -0.5944257960750252
  0.0013504531731280054
  0.07531418376834575
 -0.9019647948617265
 -1.157425026940267
  1.025079034756626


  3.491 s (6 allocations: 763.09 MiB)


In this second snippet of code we compared the "\" solver with a randomized 10000x10000 tridiagonal matrix stored as a sparse one "C" and as a full one "D". Now we see that storing the matrix as sparse led us to a way faster resolution of the problem using the base solver.

In [4]:
n=10000
f = rand(n);

#generating the low, high and main diagonal of a tridiagonal matrix
hd = rand(n-1);
ld = rand(n-1);
d = rand(n)

C = Tridiagonal(ld,d,hd)
D = Matrix(C)
display(@btime solver(C,f))
display(@btime solver(D,f))

10000-element Vector{Float64}:
  11.20452150822237
  -5.505608649084764
  -8.635341884164497
  36.953786638233964
   6.145146858873217
  -5.212940747474744
   1.6914036781374933
   0.09841245554101052
  -0.9015235907864453
   1.915375831332687
   ⋮
   0.1278842218023893
   0.4381805170804257
   1.0931076600185878
  -4.627641701733184
 -14.637187320311385
   8.510075948537498
   6.1619459688398415
 -15.212788007366823
  16.443157023023748

  162.400 μs (14 allocations: 469.12 KiB)


10000-element Vector{Float64}:
  11.204521508222376
  -5.505608649084765
  -8.6353418841645
  36.95378663823397
   6.145146858873219
  -5.212940747474745
   1.6914036781374915
   0.09841245554101197
  -0.9015235907864438
   1.915375831332683
   ⋮
   0.12788422180238904
   0.4381805170804259
   1.093107660018589
  -4.627641701733193
 -14.63718732031139
   8.510075948537507
   6.1619459688398415
 -15.212788007366829
  16.443157023023755

  3.063 s (6 allocations: 763.09 MiB)


So we can concluse that the best way to store the matrix is tridiagonal, in this way with the base solver we use less computational time.

Now we go to comparing the two solver (\ and "LinearSolve pakage") with both sparse and tridiagonal matrix.
We starting by making a function for it :  

In [9]:
function solv(Mat,b::Vector{Float64}) 
    if(size(Mat)[1]==size(Mat)[2])
        if(size(Mat)[1]!=size(b)[1])
            error("Wrong dimensions")
        end
        prob = LinearProblem(Mat, b)
        u = solve(prob, KrylovJL_CG())
        return u
    else
        error("It's not a square matrix")
    end
end

solv (generic function with 2 methods)

triying Sparse Matrix

In [10]:
@btime u=solv(A,f)

  151.051 ms (35 allocations: 392.86 KiB)


u: 10000-element Vector{Float64}:
  5.596351860995606e14
 -1.7610652769752044e14
 -1.0507163417720399e15
 -9.463276573562372e14
 -1.1895396818190275e15
  1.2551046331311982e15
 -2.1686633005469738e14
 -8.234349596359776e14
  5.0064648211997275e14
  9.036434187157792e14
  ⋮
 -9.246864008557686e14
 -1.1063979565892876e15
 -8.022426845276338e14
  1.4607306714834082e15
 -1.1074692154201142e14
  9.474125575948461e14
 -1.338973906132104e15
  6.393928949453266e14
 -1.2671547256976785e15

triying Tridiagonal Matrix

In [11]:
@btime u=solv(B,f)

  230.254 ms (33 allocations: 392.70 KiB)


u: 10000-element Vector{Float64}:
 -5.018687362140459e14
  1.579285268151015e14
  9.422597000017242e14
  8.486461845829752e14
  1.0667534701521936e15
 -1.1255507010488245e15
  1.9448103638817272e14
  7.384386700686348e14
 -4.489689417542448e14
 -8.103678821551954e14
  ⋮
  8.292387736128538e14
  9.921937683962685e14
  7.194338959045034e14
 -1.309951686840606e15
  9.931544501563395e13
 -8.496190996626424e14
  1.2007628519171705e15
 -5.733937252354348e14
  1.1363569635530658e15